In [1]:
import numpy as np
from matplotlib import pyplot as plt
import sys

# only do longitudual friction
# print("x = {0}".format(x))

def skew(vector):
    "Given a vector, return its skew symmetric matrix"
    #print("vector = {}".format(vector))
    return np.array(([0, -vector[2], vector[1]],
                     [vector[2], 0, -vector[0]],
                     [-vector[1], vector[0], 0]))
def inv_skew(matrix):
    "Given a matrix, return its inverse skew symmetric vector"
    return np.array((matrix[2][1],-matrix[2][0],matrix[1][0]))

def angle_axis(R):
    "Given a rotation matrix, return its axis and angle"
    theta = np.arccos((np.trace(R)-1)/2)
    #print("R = {0}".format(R))
    # print("R.T = {0}".format(R.T))
    # print("R-R.T = {0}".format(R-R.T))
    u = np.multiply((1/(2*np.sin(theta))),inv_skew(R-R.T))
    # print("u = {0}".format(u))
    # print("shape of u = {0}".format(np.shape(u)))
    return theta, u

def matrix_log(R):
    "Given a rotation matrix, return its matrix log"
    theta, u = angle_axis(R)
    return theta*u

def matrix_exp(theta, u):
    "Given theta and the axis, return its matrix exp"
    # print("skew(u) = {0}".format(skew(u)))
    matmul = skew(u)*skew(u)
    return 1 + np.sin(theta)*skew(u) + (1 - np.cos(theta))*matmul

def grad_h(func, nodes): # Gradiant h
    """ Modified trapezoidal integration"""
    # Pads a 0 at the end of an array
    temp = pad_along_axis(func,nodes,axis = 1) # Using roll calculate the diff (ghost node of 0)
    return (temp - np.roll(temp, 1))

def A_h(func, nodes):
    temp = pad_along_axis(func,nodes,axis = 2)
    print(temp)
    for i in range(nodes):
        print(i)
        if i == 0:
            print("hello")
            temp[:,:,i] = func[:,:,i] / 2
        
        elif i > 0 and i < nodes-1:
            print("Goodbye")
            temp[:,:,i] = (func[:,:,i] + func[:,:,i-1]) / 2
        
        elif i == nodes-1:
            print("yes?")
            temp[:,:,i] = func[:,:,i-1]/2 
            
    return(temp)
        
def pad_along_axis(array: np.ndarray, target_length: int, axis: int = 0):
    pad_size = target_length - array.shape[axis]
    if pad_size <= 0:
        return array
    npad = [(0, 0)] * array.ndim
    npad[axis] = (0, pad_size)
    return np.pad(array, pad_width=npad, mode='constant', constant_values=0)

class crossrod:
    def __init__(self, T, dt, total_length, elements, density, radius, total_external_force,G = 1E4, E = 1E5, dim = 3, **kwargs):
        # Plotting
        self.final_pos = []

        # Element Info
        self.e = elements
        self.n = self.e + 1 # nodes
        self.n_i = self.e - 1 # internal nodes

        # Initializing node mass
        self.area = np.pi * (radius**2) # Update?
        total_volume = self.area * total_length
        total_mass = density * total_volume
        self.m = np.zeros((1,self.n))
        element_mass = total_mass / self.e
        self.m[0][0] = element_mass/2
        self.m[0][1:self.n-1] = element_mass
        self.m[0][self.n-1] = element_mass/2

        # Initializing node radii
        self.r = np.full((1,self.n),radius) # Update?

        # Initializing node position
        self.pos = np.zeros((dim,self.n))
        for col in range(self.n):
            self.pos[2,col] = (total_length/self.e) * col

        # Length Info
        # UPDATE THIS AT EVERY TIME STEP
        self.l = self.pos[:,1:] - self.pos[:,:-1] # length vector
        # print("length = {0}".format(self.l))
        # print("example of a length vector = {0}".format(self.l[:,1]))
        self.l_mag = np.linalg.norm(self.l, axis = 0) # magnitude of length
        # DO NOT UPDATE THIS AT EVERY TIME STEP
        self.l_ref = self.pos[:,1:] - self.pos[:,:-1] # reference length (unstrecthed length of the rod)
        self.l_ref_mag = np.linalg.norm(self.l_ref, axis = 0) # magnitude of reference length as a scalar

        # Parameters determined by Length Info
        self.dil_fac = self.l_mag / self.l_ref_mag # dilatation factor
        self.tangents = self.l / self.l_mag # tangent vectors

        # Directors (maps from lab to material frame)
        self.directors = np.zeros((3, 3, self.e))
        # for milestone 2
        self.directors[:,:,0] = np.array(([np.sin(45), np.cos(45), 0.0],
                                          [0.0, 0.0, 1.0],
                                          [np.cos(45), -np.sin(45), 0.0]))
        self.directors[:,:,1] = np.array(([-np.sin(45), np.cos(45), 0.0],
                                          [0.0, 0.0, 1.0],
                                          [np.cos(45), np.sin(45), 0.0]))
        # print("directors = {0}".format(self.directors[:,:,0]))
        # for generalized case
        # for idx in range(self.e):
        #     self.directors[:, :, idx] = np.eye(3)

        # No forces in milestone 2
        self.forces = np.zeros((dim,self.n)) # forces INITIALIZE

        # Milestone 1 force
        # self.forces[2,self.e] = total_external_force

        self.vel = np.zeros((dim,self.n)) # velocities
        self.ang_vel = np.zeros((dim,self.e)) # angular velocities

        # Shear/stretch diagonal matrix INITIALIZE INPUT FROM MATERIAL PROPERTIES
        self.S_hat = np.zeros((3,3,self.e))
        # print("S_hat = {0}".format(self.S_hat))
        alpha_c = 4./3. # shape factor
        self.S_hat[0,0,:] = alpha_c * G * self.area
        self.S_hat[1,1,:] = alpha_c * G * self.area
        self.S_hat[2,2,:] = E * self.area
        # print("S_hat = {0}".format(self.S_hat))

        # Moment of inertia diagonal matrix
        self.I = np.zeros((3,3,self.e))
        self.I[0,0,:] = self.area**2 / 4 * np.pi
        self.I[1,1,:] = self.area**2 / 4 * np.pi
        self.I[2,2,:] = self.area**2 / 4 * np.pi * 2

        # Bend diagonal matrix INITIALIZE INPUT FROM MATERIAL PROPERTIES
        self.B = np.zeros((3,3,self.n_i))
        self.B[0,0,:] = E * self.I[0,0,1:]
        self.B[1,1,:] = E * self.I[1,1,1:]
        self.B[2,2,:] = G * self.I[2,2,1:]

        # J diagonal matrix.
        # **** if broken code, there might be some difference between dJ^ and J^
        # here i assume J is pI from dJ = pIds
        self.J = np.zeros((3,3,self.e))
        self.J[0,0,:] = density * self.I[0,0,:]
        self.J[1,1,:] = density * self.I[1,1,:]
        self.J[2,2,:] = density * self.I[2,2,:]

        # kappa here
        self.kappa = np.zeros((3,3,self.n_i)) # NOT SURE IF THIS SHAPE IS CORRECT
        #print("initialized kappa = {0}".format(self.kappa))
        for idx in range(self.n_i):
            self.kappa[:,:,idx] = -(matrix_log(np.matmul(self.directors[:,:,(idx+1)],self.directors[:,:,idx].T)))/self.l_mag[idx]
            # print("kappa = {0}".format(self.kappa))

        # shear/stress strain
        self.sigma = self.dil_fac * self.tangents - self.directors[2,:,:]

        # Governing Equations
        # pos += vel * dt # Equation 1
        # dv_dt = (grad_h(S_hat @ s / dil_fac) + f) / m # Equation 3

        for x in np.arange(0,T+dt,dt):
            self.pos, self.vel = self.position_verlet(dt, self.pos, self.vel)
            self.directors, self.ang_vel = self.angular_verlet(dt, self.directors, self.ang_vel)
            self.force_update(self.pos)
            self.bend_update(self.directors)
            self.final_pos.append(self.pos[2,-1])

    def position_verlet(self, dt, x, v):
        # x = position & v = velocity
        temp_x = x + 0.5*dt*v
        v_n = v + dt * self.force_rule(temp_x)
        x_n = temp_x + 0.5 * dt * v_n
        return x_n, v_n

    def angular_verlet(self, dt, Q, w):
        # Q = directors & w = angular velocity
        # print("w = {0}".format(w))
        # print("Q = {0}".format(Q))
        temp_Q = np.zeros((3,3,self.e))
        for idx in range(self.e):
            temp_Q[:,:,idx] = matrix_exp((-dt/2),w[:,idx]) * Q[:,:,idx]
        #print("temp_Q = {0}".format(temp_Q))
        w_n = w + dt * self.bend_rule(temp_Q)
        #print("w_n = {0}".format(w_n))
        Q_n = np.zeros((3,3,self.e))
        for idx in range(self.e):
            Q_n[:,:,idx] = matrix_exp((-dt/2),w_n[:,idx]) * temp_Q[:,:,idx]
        #print("Q_n = {0}".format(Q_n))
        return Q_n, w_n

    def force_rule(self, temp_pos):
        # First update
        self.force_update(temp_pos)

        matmul = np.zeros((3,self.e))
        matmul = np.einsum('jil, jkl, kl -> il ', self.directors, self.S_hat, self.sigma)

        self.internal_force = grad_h(matmul / self.dil_fac, self.n)
        print("internal_force shape = {0}".format())

        dv_dt = (self.internal_force + self.forces)  / self.m
        return dv_dt

    def bend_rule(self, temp_Q):
        # First update
        self.bend_update(temp_Q)

        # Governing equation 4 with assumptions
        matmulf = np.zeros((3,3,self.n_i))
        for idx in range(self.n_i):
            matmulf[:,:,idx] = self.B[:,:,idx] * self.kappa[:,:,idx]
        # matmul = np.einsum('ijk,jk->ik',self.B,self.kappa)

        termOne = matmulf / self.epsilon**3
        
        termTwo
        
        self.bend_twist_internal_couple = grad_h(termOne, self.e) + 
        # print("bend_twist_internal_couple shape = {0}".format(np.shape(self.bend_twist_internal_couple)))

        matmulfirst = np.zeros((3,3,self.e))
        matmulfirst = np.einsum('ijk,jk->ik',temp_Q,self.tangents)
        # print("matmulfirst = {0}".format(np.shape(matmulfirst)))
        # print("matmulfirst = {0}".format(matmulfirst))

        matmulsecond = np.zeros((3,3,self.e))
        matmulsecond = np.einsum('ijk,jk->ik',self.S_hat,self.sigma)
        #print("matmulsecond = {0}".format(np.shape(matmulsecond)))
        # print("matmulsecond = {0}".format(matmulsecond))

        self.shear_stretch_internal_couple = np.zeros((3,3,self.e))
        for idx in range(self.e):
            self.shear_stretch_internal_couple[:,idx] = np.cross(matmulfirst[:,idx],matmulsecond[:,idx]) # *self.l_ref_mag
        print("shear_stretch_internal_couple = {0}".format(self.shear_stretch_internal_couple))
        print("shear_stretch_internal_couple shape = {0}".format(np.shape(self.shear_stretch_internal_couple)))
        print("J shape = {0}".format(np.shape(self.J)))
        
        dw_dt = (self.bend_twist_internal_couple + self.shear_stretch_internal_couple)  / self.J
        return dw_dt

    def force_update(self, temp_pos):
        # Constrain 1st and last node position and velocity
        temp_pos[:,0] = 0
        temp_pos[:,-1] = 0
        self.vel[:,0] = 0
        self.vel[:,-1] = 0

        # Update Length
        self.l = temp_pos[:,1:] - temp_pos[:,:-1]
        self.l_mag = np.linalg.norm(self.l, axis = 0)

        # Update dilatation factor
        self.dil_fac = self.l_mag / self.l_ref_mag

        # Update tangents
        self.tangents = self.l / self.l_mag

        # Update shear/stress strain
        self.s = self.dil_fac * self.tangents - self.directors[2,:,:]
        pass

    def bend_update(self, temp_Q):
        """ Only Kappa depends on the directors """
        for idx in range(self.n_i):
            self.kappa[:,:,idx] = -(matrix_log(np.matmul(temp_Q[:,:,(idx+1)],temp_Q[:,:,idx].T)))/self.l[:,idx]





In [2]:
F = 15
E = 1E5
R = .25
A = np.pi * R**2
L = 3
T = 100
dt = 3E-4

In [5]:
test = crossrod(T = T, dt = dt, total_length = L, elements = 5, density = 5E3, radius = R, total_external_force = F)
print("position = {0}".format(test.pos))
real_strain = (F*L)/(E*A-F)
print(real_strain)
print(np.average(test.final_pos))
plt.plot(np.arange(0,T+dt,dt),test.final_pos)
plt.show()

IndexError: Replacement index 0 out of range for positional args tuple